In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics.pairwise import cosine_similarity
import string
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import hstack
import spacy
import scipy.sparse as sp
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import warnings
from collections import Counter
from itertools import islice

warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip install textstat -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.8 MB/s eta 0:00:00


In [3]:
# %%capture
!pip install language-tool-python -q
# !python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.8 MB/s eta 0:00:00


In [4]:
import language_tool_python
import textstat

tool = language_tool_python.LanguageTool('en-US', remote_server='https://api.languagetool.org')

In [5]:
train_df = pd.read_csv('/kaggle/input/pan-cief/train.csv')
test_df = pd.read_csv('/kaggle/input/pan-cief/val.csv')

## helper functions

In [6]:
def vocabSize(sentence):

    doc = nlp(sentence.lower())
    tokens = set([token.text for token in doc if not token.is_punct])
    vocab_size = len(tokens)

    return vocab_size

In [7]:
def sentence_complexity(sentence):

  flesch_score = textstat.flesch_reading_ease(sentence) # Flesch Reading Ease: Measures the readability of a sentence. Higher scores indicate easier readability.
  fk_grade_level = textstat.flesch_kincaid_grade(sentence) # Flesch-Kincaid Grade Level: Indicates the grade level required to understand the text. Higher scores mean more complex text.
  gunning_fog = textstat.gunning_fog(sentence) # Gunning Fog Index: Measures the readability of the text, considering sentence length and complex words (3+ syllables). Higher values indicate more complex text.
  smog_index = textstat.smog_index(sentence)  # SMOG Index: Estimates the years of education required to understand the text. Higher values indicate more complex text.
  composite_score = (flesch_score * 0.2 + fk_grade_level * 0.3 + gunning_fog * 0.3 + smog_index * 0.2)
    # print(f"Composite Sentence Complexity Score(In scale of 1 - 20): {composite_score}")

  return composite_score

In [8]:
def grammatical_mistakes(sentence):

  mistakes = len(tool.check(sentence))

  return mistakes

In [9]:
def punctuation_count(paragraph):
    # Create a dictionary to store counts of each punctuation mark
    punctuation_counts = {p: 0 for p in string.punctuation}

    # Count each punctuation mark in the paragraph
    for char in paragraph:
        if char in string.punctuation:
            punctuation_counts[char] += 1

    # Calculate the total number of punctuation marks
    total_punctuation = sum(punctuation_counts.values())

    return total_punctuation

In [10]:
def sentence_length_difference(paragraph):

    # Split the paragraph into sentences using common sentence delimiters
    import re
    sentences = re.split(r'[.!?]', paragraph)

    # Remove empty strings and whitespace from the list of sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Calculate the length of each sentence
    sentence_lengths = [len(sentence.split()) for sentence in sentences]

    # Calculate the difference between the smallest and largest sentence length
    length_difference = max(sentence_lengths) - min(sentence_lengths) if sentence_lengths else 0

    return length_difference

In [11]:
def type_token_ratio(text):
    """
    Calculates the Type-Token Ratio (TTR) for vocabulary diversity.
    """
    words = text.split()
    unique_words = set(words)
    ttr = len(unique_words) / len(words) if len(words) > 0 else 0
    return ttr

In [12]:
train_df['character_count'] = train_df['text'].str.len()
test_df['character_count'] = test_df['text'].str.len()

In [13]:
train_df['word_count'] = train_df['text'].apply(lambda x: len(x.split()))
test_df['word_count'] = test_df['text'].apply(lambda x: len(x.split()))

In [14]:
train_df['sentence_count'] = train_df['text'].apply(lambda x: len(re.split(r'[.!?]', x.strip())) - 1)
test_df['sentence_count'] = test_df['text'].apply(lambda x: len(re.split(r'[.!?]', x.strip())) - 1)

In [15]:
train_df['paragraph_count'] = train_df['text'].apply(lambda x: len(x.split("\n")))
test_df['paragraph_count'] = test_df['text'].apply(lambda x: len(x.split("\n")))

In [16]:
stop_words = set(stopwords.words('english'))
train_df['stopword_count'] = train_df['text'].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))
test_df['stopword_count'] = test_df['text'].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))

In [17]:
train_df['unique_word_count'] = train_df['text'].apply(lambda x: len(set(x.split())))
test_df['unique_word_count'] = test_df['text'].apply(lambda x: len(set(x.split())))

In [18]:
nlp = spacy.load("en_core_web_sm")

def pos_counts(text):
    doc = nlp(text)
    pos_count_dict = {}
    for token in doc:
        pos = token.pos_
        pos_count_dict[pos] = pos_count_dict.get(pos, 0) + 1
    return pos_count_dict

train_df['pos_counts'] = train_df['text'].apply(pos_counts)
test_df['pos_counts'] = test_df['text'].apply(pos_counts)

In [19]:
from textblob import TextBlob

train_df['sentiment_polarity'] = train_df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
train_df['sentiment_subjectivity'] = train_df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

test_df['sentiment_polarity'] = test_df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
test_df['sentiment_subjectivity'] = test_df['text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [20]:
discourse_markers = ["however", "therefore", "although", "nevertheless", "hence"]

def count_discourse_markers(text):
    return sum(text.lower().count(marker) for marker in discourse_markers)

train_df['discourse_marker_count'] = train_df['text'].apply(count_discourse_markers)
test_df['discourse_marker_count'] = test_df['text'].apply(count_discourse_markers)

In [21]:
train_df['vocab_size'] = train_df['text'].apply(vocabSize)
train_df['sentence_complexity'] = train_df['text'].apply(sentence_complexity)
# train_df['grammatical_mistakes'] = train_df['text'].apply(grammatical_mistakes)
train_df['punctuation_count'] = train_df['text'].apply(punctuation_count)
train_df['sentence_length_difference'] = train_df['text'].apply(sentence_length_difference)
train_df['type_token_ratio'] = train_df['text'].apply(type_token_ratio)

In [22]:
test_df['vocab_size'] = test_df['text'].apply(vocabSize)
test_df['sentence_complexity'] = test_df['text'].apply(sentence_complexity)
# test_df['grammatical_mistakes'] = test_df['text'].apply(grammatical_mistakes)
test_df['punctuation_count'] = test_df['text'].apply(punctuation_count)
test_df['sentence_length_difference'] = test_df['text'].apply(sentence_length_difference)
test_df['type_token_ratio'] = test_df['text'].apply(type_token_ratio)

In [23]:
from scipy.stats import entropy

def word_entropy(text):
    doc = nlp(text)
    words = [t.lemma_.lower() for t in doc if t.is_alpha]
    freqs = list(Counter(words).values())
    return entropy(freqs) if freqs else 0

def flesch_reading_ease(text):
    try:
        return textstat.flesch_reading_ease(text)
    except:
        return 0

def gzip_ratio(text):
    import gzip
    compressed = len(gzip.compress(text.encode('utf-8')))
    return compressed / len(text) if len(text) > 0 else 0


In [24]:
train_df['word_entropy'] = train_df['text'].apply(word_entropy)
train_df['flesch_reading_ease'] = train_df['text'].apply(flesch_reading_ease)
train_df['gzip_ratio'] = train_df['text'].apply(gzip_ratio)

In [25]:
test_df['word_entropy'] = test_df['text'].apply(word_entropy)
test_df['flesch_reading_ease'] = test_df['text'].apply(flesch_reading_ease)
test_df['gzip_ratio'] = test_df['text'].apply(gzip_ratio)

In [26]:
def negation_frequency(text):
    doc = nlp(text)
    negations = [t for t in doc if t.dep_ == "neg" or t.lemma_.lower() in ["not", "no", "never", "none", "n't"]]
    total_words = len([t for t in doc if t.is_alpha])
    return len(negations) / (total_words + 1e-5)

def question_statement_ratio(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    if not sentences:
        return 0
    question_count = sum(1 for s in sentences if s.text.strip().endswith("?"))
    statement_count = sum(1 for s in sentences if s.text.strip().endswith("."))
    return question_count / (statement_count + 1e-5)

def clause_to_sentence_ratio(text):
    doc = nlp(text)
    sentences = list(doc.sents)
    if not sentences:
        return 0
    clause_markers = ("mark", "advcl", "ccomp", "xcomp", "acl", "relcl", "conj")
    clause_count = sum(1 for t in doc if t.dep_ in clause_markers)
    return clause_count / len(sentences)

def modal_verb_frequency(text):
    doc = nlp(text)
    modals = {"can", "could", "may", "might", "shall", "should", "will", "would", "must"}
    modal_count = sum(1 for t in doc if t.lemma_.lower() in modals)
    total_words = len([t for t in doc if t.is_alpha])
    return modal_count / (total_words + 1e-5)

def pronoun_ratio(text):
    doc = nlp(text)
    pronouns = [t for t in doc if t.pos_ == "PRON"]
    total_words = len([t for t in doc if t.is_alpha])
    return len(pronouns) / (total_words + 1e-5)



In [27]:
from scipy.stats import entropy

def pos_diversity(text):
    doc = nlp(text)
    pos_tags = [t.pos_ for t in doc if t.is_alpha]
    counts = Counter(pos_tags)
    return entropy(list(counts.values())) if counts else 0

def pos_ngram_frequency_deviation(text, ref_pos_ngrams=None, n=2):
    doc = nlp(text)
    pos_tags = [t.pos_ for t in doc if t.is_alpha]
    ngrams = ["_".join(pos_tags[i:i+n]) for i in range(len(pos_tags)-n+1)]
    counts = Counter(ngrams)
    total = sum(counts.values())
    if total == 0 or ref_pos_ngrams is None:
        return 0
    # Normalize
    probs = {k: v/total for k, v in counts.items()}
    # Compute chi-square-like deviation
    deviation = 0
    for k, p in probs.items():
        expected = ref_pos_ngrams.get(k, 1e-6)
        deviation += ((p - expected) ** 2) / expected
    return deviation

def hapax_legomena_ratio(text):
    doc = nlp(text)
    words = [t.lemma_.lower() for t in doc if t.is_alpha]
    if not words:
        return 0
    freqs = Counter(words)
    hapax = sum(1 for w, f in freqs.items() if f == 1)
    return hapax / len(freqs)


In [28]:
train_df['negation_freq'] = train_df['text'].apply(negation_frequency)
train_df['question_stmt_ratio'] = train_df['text'].apply(question_statement_ratio)
train_df['clause_sentence_ratio'] = train_df['text'].apply(clause_to_sentence_ratio)
train_df['modal_freq'] = train_df['text'].apply(modal_verb_frequency)
train_df['pronoun_ratio'] = train_df['text'].apply(pronoun_ratio)
train_df['pos_diversity'] = train_df['text'].apply(pos_diversity)
train_df['hapax_ratio'] = train_df['text'].apply(hapax_legomena_ratio)

In [29]:
test_df['negation_freq'] = test_df['text'].apply(negation_frequency)
test_df['question_stmt_ratio'] = test_df['text'].apply(question_statement_ratio)
test_df['clause_sentence_ratio'] = test_df['text'].apply(clause_to_sentence_ratio)
test_df['modal_freq'] = test_df['text'].apply(modal_verb_frequency)
test_df['pronoun_ratio'] = test_df['text'].apply(pronoun_ratio)
test_df['pos_diversity'] = test_df['text'].apply(pos_diversity)
test_df['hapax_ratio'] = test_df['text'].apply(hapax_legomena_ratio)

In [30]:
train_df.to_csv('train.csv',index=False)

In [31]:
test_df.to_csv('test.csv',index=False)